<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Michelle Avendaño
- Nombre de alumno 2: Claudia Navarro


### **Link de repositorio de GitHub:** `https://github.com/ClaudiaRayen/MDS7202-Laboratorios/tree/Lab7`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 76.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [6]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/online_retail_II_cleaned.pickle'
except:
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [8]:
df_retail = pd.read_pickle("/content/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [9]:
def custom_features(dataframe_in):
    # Agrupar los datos por cliente
    grouped = dataframe_in.groupby('Customer ID')

    features = grouped.agg(
        Length=pd.NamedAgg(column='InvoiceDate', aggfunc=lambda x: (x.max() - x.min()).days),
        Recency=pd.NamedAgg(column='InvoiceDate', aggfunc=lambda x: (dataframe_in['InvoiceDate'].max() - x.max()).days),
        Monetary=pd.NamedAgg(column='Price', aggfunc='mean'),
        Frequency=pd.NamedAgg(column='InvoiceDate', aggfunc='nunique'),
        Periodicity=pd.NamedAgg(column='InvoiceDate', aggfunc=lambda x: (x - x.shift(1)).dt.days.std() if len(x) > 1 else None)
    )

    return features

# Calcular las características LRMFP
LRMFP = custom_features(df_retail)
LRMFP

,Length,Recency,Monetary,Frequency,Periodicity
Customer ID,,,,,
12346.0,196,164,6.253333,11,21.724076
12347.0,37,2,2.295070,2,4.422346
12348.0,0,73,0.719500,1,0.000000
12349.0,181,42,8.581765,3,16.200990
12351.0,0,10,2.355238,1,0.000000
...,...,...,...,...,...
18283.0,275,17,2.199401,6,11.783701
18284.0,0,66,3.253214,1,0.000000
18285.0,0,295,8.350000,1,0.000000


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [10]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Calculo de maximo y minimo para cada columna
        self.min_ = X.min()
        self.max_ = X.max()
        return self

    def transform(self, X, y=None):
        # aplicar la ecuacion
        return (X - self.min_) / (self.max_ - self.min_)

# Excluir "Customer Id" de la transformacion
columns_to_scale = [col for col in LRMFP.columns if col != 'Customer Id']

# Aplicar minmax en las columnas
transformer = ColumnTransformer(
    transformers=[('minmax', MinMax(), columns_to_scale)],
    remainder='passthrough'
)

# Pipeline para aplicar la transformacion
pipeline = Pipeline(steps=[('scaler', transformer)])

#dataframe resultanrte
LRMFP_transformed = pipeline.fit_transform(LRMFP)
LRMFP_transformed_df = pd.DataFrame(LRMFP_transformed, columns=LRMFP.columns)
LRMFP_transformed_df

,Length,Recency,Monetary,Frequency,Periodicity
0,0.525469,0.439678,0.000571,0.049261,0.119910
1,0.099196,0.005362,0.000210,0.004926,0.024410
2,0.000000,0.195710,0.000066,0.000000,0.000000
3,0.485255,0.112601,0.000783,0.009852,0.089425
4,0.000000,0.026810,0.000215,0.000000,0.000000
...,...,...,...,...,...
4309,0.737265,0.045576,0.000201,0.024631,0.065043
4310,0.000000,0.176944,0.000297,0.000000,0.000000
4311,0.000000,0.790885,0.000762,0.000000,0.000000
4312,0.662198,0.297587,0.000390,0.004926,0.167819


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

Luego del Paso 1, aparecen NA en Periodicity debido a registros donde hubieron solo una visita, es por esto que se decide eliminar estos NA pues una visita no da información importante.

In [43]:
# Paso 1: Usando FunctionTransformer para convertir la función en un transformador
feature_extractor = FunctionTransformer(custom_features, validate=False)

#Imputador: Necesario debido a que custom_features provoca NAN en "Periodicity"
imputer = SimpleImputer(strategy='constant', fill_value=None)

# Paso 2: Como las columnas a escalar serían todas las obtenidas de custom_features, no necesitamos excluir ninguna aquí.
transformer = ColumnTransformer(
    transformers=[('minmax', MinMax(), slice(0, -1))],  #slice para seleccionar todas las columnas
    remainder='passthrough'
)

#Paso 3:T-SNE
tsne = TSNE(n_components=2)

#Se crea el pipline
pipeline = Pipeline([
    ('features', feature_extractor),
    ('impute', imputer),  # Paso de elinar Nan
    ('scaler', transformer),
    ('dimension_reduction', tsne)
])


In [58]:
# Aplicar el pipeline a los datos
LRMFP_transformed = pipeline.fit_transform(df_retail)

# Crear un DataFrame con los componentes transformados
df_transformed = pd.DataFrame(data=LRMFP_transformed, columns=['Componente 1', 'Componente 2'])

# Crear el gráfico de dispersión con Plotly
fig = px.scatter(df_transformed, x='Componente 1', y='Componente 2', title='T-SNE 2D')
fig.update_layout(width=800, height=700)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [73]:
import random
random.seed(321)

# Definiendo el pipeline
kmeans_pipeline = Pipeline([
    ('features', feature_extractor),
    ('impute', imputer),
    ('scaler', transformer),
    ('kmeans', KMeans(n_clusters=1))  # Valor placeholder
])


#Método del codo

inertia_values = []

# Estimar la inercia para cada k
for k in range(1, 21):
    kmeans_pipeline.named_steps['kmeans'].set_params(n_clusters=k)
    kmeans_pipeline.fit(df_retail)
    inertia_values.append(kmeans_pipeline.named_steps['kmeans'].inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

In [72]:
# Crear un DataFrame con los valores de inercia
df_inertia = pd.DataFrame({'Número de Cluster': range(1, 21), 'Inercia': inertia_values})

# Crear el gráfico del método del codo con Plotly
fig = px.line(df_inertia, x='Número de Cluster', y='Inercia', markers=True, title='Método del Codo')
fig.update_traces(mode='markers+lines')  # Para mostrar puntos y líneas
fig.update_layout(width=800, height=700)
fig.show()

A partir del gráfico, se puede observar que la inercia disminuye rápidamente para valores bajos de clústeres y luego comienza a nivelarse. Aunque no hay un "codo" pronunciado, parece que hay un punto de inflexión alrededor de 5 clústeres, donde la tasa de disminución de la inercia comienza a ralentizarse. A partir de ese punto, agregar más clústeres solo disminuye ligeramente la inercia.

Dicho esto, es razonable escoger 5 clústeres como el número óptimo de clústeres para el KMeans en este caso, ya que ofrece un equilibrio entre minimizar la inercia y no sobre-segmentar los datos.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [74]:
random.seed(321)
kmeans_pipeline.named_steps['kmeans'].set_params(n_clusters= 5)
clusters = kmeans_pipeline.fit_predict(df_retail)
df_clusters = LRMFP.copy()
df_clusters['Cluster'] = clusters

# Calculando promedios agrupados por clúster
cluster_means = df_clusters.groupby('Cluster').mean()
cluster_means

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



,Length,Recency,Monetary,Frequency,Periodicity
Cluster,,,,,
0,35.895551,124.655222,16.683191,3.744681,1.072173
1,253.607143,57.790179,4.959600,2.901786,46.692657
2,226.181009,61.048961,3.556213,3.654303,24.758571
3,276.276596,48.234043,3.925057,2.617021,103.589834
4,216.242121,57.859339,3.339107,6.302075,11.578401


Los cluster no dan información tan importante, pues se parecen entre si. El cluster 0 es el más diferente que representa a los clientes menos leales, sin embargo, los otros se comportan moderadamente en la mayoría de las variables, siendo Periodicity la variable que más cambia por cluster.


Cluster 0:


*   Tiempo entre la primera y la última visita (Length) es relativamente bajo (35.90 días).
*Hace mucho tiempo que el cliente realizó su última compra (Recency es alto).
*El gasto promedio por visita (Monetary) es bajo.
*La frecuencia de visitas (Frequency) es moderada.
*La periodicidad es baja, lo que indica que los clientes en este clúster no visitan la tienda con regularidad.

Cluster 1:

* Length es alto, lo que indica que los clientes en este clúster han estado visitando durante un largo período.
*Recency es moderado, lo que sugiere que han realizado compras recientemente.
*El gasto promedio por visita (Monetary) es bajo.
*La frecuencia de visitas (Frequency) es moderada.
*La periodicidad es alta, lo que indica que los clientes en este clúster visitan la tienda con regularidad.

Cluster 2:
* Similar a Cluster 1 en términos de Length, Recency y Monetary.
*La frecuencia de visitas (Frequency) es más alta que en Cluster 1.
*La periodicidad es moderada.

Cluster 3:
* Length es alto.
*Recency es moderado.
*El gasto promedio por visita (Monetary) es moderado.
*La frecuencia de visitas (Frequency) es más baja que en otros clústeres.
*La periodicidad es muy alta, lo que indica visitas regulares.

Cluster 4:
* Length es alto.
*Recency es moderado.
*El gasto promedio por visita (Monetary) es bajo.
*La frecuencia de visitas (Frequency) es alta, lo que sugiere clientes frecuentes.
*La periodicidad es moderada.


Los clientes en Cluster 0 tienen una baja frecuencia de visita, gastan poco y no visitan la tienda con regularidad. Pueden ser clientes menos leales.
Los clientes en Cluster 3 son los que han estado visitando durante más tiempo y visitan la tienda con la mayor regularidad, pero gastan moderadamente.
Los clientes en Cluster 4 también tienen una alta frecuencia de visitas y un gasto bajo, lo que podría indicar compradores ocasionales o de bajo presupuesto.
Los clústeres 1 y 2 tienen características similares en términos de Length y Recency, pero difieren en Frequency y Periodicity. Esto podría representar dos grupos diferentes de clientes con diferentes patrones de compra.








**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

Es evidente que el Cluster 0, que representa a clientes menos leales, es el grupo más grande en términos de la cantidad de puntos. Le sigue el Cluster 4, compuesto por clientes con una alta periodicidad en sus visitas a la tienda.

En relación a la pregunta planteada, se observa que los clústeres no están claramente separados en la visualización. Esto se debe al uso del algoritmo K-Means para la segmentación, el cual puede no ser el más apropiado para el tipo de distribución que presenta T-SNE en los datos. La forma de los datos sugiere que otro algoritmo, como DBSCAN, podría ser más adecuado. K-Means funciona mejor en datos con estructuras esféricas, mientras que T-SNE puede generar representaciones no lineales que dificultan la separación clara en clústeres.

In [75]:
# Agregar los resultados a un nuevo DataFrame
df_TSNE = pd.DataFrame(data=LRMFP_transformed, columns=['Componente 1', 'Componente 2'])
df_TSNE['Cluster'] = clusters

# Crear el gráfico de dispersión con Plotly
fig = px.scatter(df_TSNE, x='Componente 1', y='Componente 2', color='Cluster', title='T-SNE 2D con KMeans Clusters', opacity=0.5, color_continuous_scale='viridis')
fig.update_layout(width=800, height=700)
fig.show()

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>